In [1]:
%cd /content/sample_data

/content/sample_data


In [ ]:
#!pip install trtorch 

In [ ]:
#!git clone -b master https://github.com/nvidia/TensorRT TensorRT

In [ ]:
#%cd TensorRT

In [ ]:
#!git submodule update --init --recursive

In [ ]:
#!./docker/build.sh --file docker/ubuntu-18.04.Dockerfile --tag tensorrt-ubuntu18.04-cuda11.4

In [ ]:
!gdown --id 1q-lnbnWv9pdM3xTavvAvhP57oW5z88yN

Downloading...
From: https://drive.google.com/uc?id=1q-lnbnWv9pdM3xTavvAvhP57oW5z88yN
To: /content/sample_data/checkpoint-162225.zip
100% 1.17G/1.17G [00:07<00:00, 154MB/s]


In [ ]:
!unzip /content/sample_data/checkpoint-162225.zip

Archive:  /content/sample_data/checkpoint-162225.zip
replace checkpoint-162225/preprocessor_config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
#!pip install onnx onnxruntime

In [104]:
!pip install onnxruntime-gpu

     |████████████████████████████████| 95.6 MB 1.4 MB/s 


In [ ]:
!pip install transformers

In [2]:
import soundfile as sf
import numpy as np

In [3]:
from time import time

In [4]:
import io
import numpy as np

from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

In [5]:
model = Wav2Vec2ForCTC.from_pretrained("/content/sample_data/checkpoint-162225").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("/content/sample_data/checkpoint-162225")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
audio, sr = sf.read("/content/sample_data/fp6ABAjcXPI.0008.wav")

In [ ]:
np.shape(audio)

(31664,)

In [7]:
inputs = processor(audio, sampling_rate=16_000, return_tensors="pt", padding=True).to("cuda")

In [ ]:
np.shape(inputs.input_values)

torch.Size([1, 31664])

In [8]:
batch_size = 1
size_audio = 16000

In [9]:
x = torch.randn(batch_size, size_audio , requires_grad=True).to("cuda")

In [ ]:
type(x)

torch.Tensor

In [10]:
torch_out = model(x)

In [ ]:
type(torch_out.logits)

torch.Tensor

In [11]:
torch.onnx.export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "super_resolution.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size', 1: "size_audio"},    # variable length axes
                                'output' : {0 : 'batch_size', 1: "size_audio"}})

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:538: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_weights.size() != (bsz * self.num_heads, tgt_len, src_len):
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:575: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_output.size() != (bsz * self.num_heads, tgt_len, self.head_dim):


In [12]:
import onnx

onnx_model = onnx.load("super_resolution.onnx")
onnx.checker.check_model(onnx_model)

In [13]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("super_resolution.onnx")

/usr/local/lib/python3.7/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:353: UserWarning: Deprecation warning. This ORT build has ['CUDAExecutionProvider', 'CPUExecutionProvider'] enabled. The next release (ORT 1.10) will require explicitly setting the providers parameter (as opposed to the current behavior of providers getting set/registered by default based on the build flags) when instantiating InferenceSession.For example, onnxruntime.InferenceSession(..., providers=["CUDAExecutionProvider"], ...)
  "based on the build flags) when instantiating InferenceSession."


In [14]:
onnxruntime.get_device()

'GPU'

In [15]:
def to_numpy(tensor):
    start = time()
    result = tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    end = time()
    print("time to numpy %f" %(end-start))
    return result
# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out.logits), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

time to numpy 0.001126
time to numpy 0.000237
Exported model has been tested with ONNXRuntime, and the result looks good!


In [74]:
ort_outs

[array([[[ -1.5236202,  -5.859751 , -10.887591 , ...,  -2.9987051,
          -23.808561 ,  11.289816 ],
         [ -1.877117 ,  -6.2698994, -12.296979 , ...,  -3.6036615,
          -25.525177 ,  12.7112465],
         [ -1.8982913,  -6.290418 , -12.582239 , ...,  -3.677633 ,
          -25.96843  ,  13.105109 ],
         ...,
         [ -2.102715 ,  -6.1728196, -12.379694 , ...,  -4.001382 ,
          -26.481075 ,  12.929312 ],
         [ -1.8755506,  -6.018666 , -12.206705 , ...,  -4.291876 ,
          -25.908623 ,  12.990101 ],
         [ -1.6166798,  -5.806743 , -11.693392 , ...,  -4.397772 ,
          -25.472271 ,  12.503278 ]]], dtype=float32)]

In [75]:
torch_out

CausalLMOutput([('logits',
                 tensor([[[ -1.5236,  -5.8598, -10.8876,  ...,  -2.9987, -23.8085,  11.2898],
                          [ -1.8771,  -6.2699, -12.2970,  ...,  -3.6037, -25.5252,  12.7113],
                          [ -1.8983,  -6.2904, -12.5822,  ...,  -3.6776, -25.9684,  13.1051],
                          ...,
                          [ -2.1027,  -6.1728, -12.3797,  ...,  -4.0014, -26.4810,  12.9293],
                          [ -1.8756,  -6.0187, -12.2067,  ...,  -4.2919, -25.9086,  12.9901],
                          [ -1.6167,  -5.8067, -11.6934,  ...,  -4.3978, -25.4723,  12.5033]]],
                        device='cuda:0', grad_fn=<AddBackward0>))])

In [ ]:
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(inputs.input_values)}
ort_outs = ort_session.run(None, ort_inputs)
img_out_y = ort_outs[0]

In [ ]:
img_out_y

array([[[  3.0382507,   2.5965843,  -5.1624994, ...,   2.8944583,
         -16.373123 ,   6.0509295],
        [ -2.713927 ,  -3.764171 ,  -7.221685 , ...,  -2.4261508,
         -16.75057  ,   7.7543693],
        [  0.3424537,  -2.1360927,  -7.2895823, ...,   2.083418 ,
         -15.00036  ,   7.149744 ],
        ...,
        [ -1.8987188,  -5.024401 , -11.948967 , ...,  -3.897209 ,
         -24.465649 ,  14.01545  ],
        [ -1.9270054,  -4.8025517, -11.571903 , ...,  -3.082398 ,
         -23.997757 ,  13.85802  ],
        [ -2.5925958,  -5.4269423, -11.90817  , ...,  -3.3396032,
         -24.5448   ,  13.983185 ]]], dtype=float32)

In [ ]:
predicted_ids = np.argmax(img_out_y, axis=-1)
transcription = processor.decode(predicted_ids[0])
print(transcription)

maklumkan 5 minit setiap


In [ ]:
with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [ ]:
logits

tensor([[[  3.0382,   2.5966,  -5.1625,  ...,   2.8945, -16.3731,   6.0509],
         [ -2.7139,  -3.7642,  -7.2217,  ...,  -2.4261, -16.7506,   7.7544],
         [  0.3425,  -2.1361,  -7.2896,  ...,   2.0834, -15.0004,   7.1497],
         ...,
         [ -1.8987,  -5.0244, -11.9490,  ...,  -3.8972, -24.4656,  14.0154],
         [ -1.9270,  -4.8026, -11.5719,  ...,  -3.0824, -23.9978,  13.8580],
         [ -2.5926,  -5.4270, -11.9082,  ...,  -3.3396, -24.5448,  13.9832]]])

In [ ]:
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.decode(predicted_ids[0])
print(transcription)

maklumkan 5 minit setiap


In [49]:
audio, sr = sf.read("/content/sample_data/fp6ABAjcXPI.0019.wav")
inputs = processor(audio, sampling_rate=16_000, return_tensors="pt", padding=True).to("cuda")

In [47]:
def infer_with_onnx(inputs):
  start_time = time()
  ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(inputs.input_values)}
  ort_time = time()
  print(f"ort_time: {ort_time - start_time}")
  ort_outs = ort_session.run(None, ort_inputs)
  logits = ort_outs[0]
  predict_time = time()
  predicted_ids = np.argmax(logits, axis=-1)
  transcription = processor.decode(predicted_ids[0])
  end_time = time()
  print(f"{transcription} , pred_time: {predict_time - start_time}")
  return transcription

In [48]:
a = infer_with_onnx(inputs)

time to numpy 0.000257
ort_time: 0.0004115104675292969
dengan penupekerti bersama menya , pred_time: 0.15183639526367188


In [41]:
def infer_with_normal(input):
  start_time = time()
  with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits
  predict_time = time()
  predicted_ids = torch.argmax(logits, axis=-1)
  transcription = processor.decode(predicted_ids[0])
  end_time = time()
  print(f"{transcription} , pred_time: {predict_time - start_time}")
  return transcription

In [45]:
b = infer_with_normal(inputs)

dengan penupekerti bersama menya , pred_time: 0.060021162033081055


In [94]:
c = to_numpy(inputs.input_values.to("cuda"))

In [98]:
np.shape(inputs.input_values[0])

torch.Size([34576])